**GAN**

In [5]:
!git clone https://github.com/AvonYangXX1/DreamWalker.git
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split, KFold

fatal: destination path 'DreamWalker' already exists and is not an empty directory.


In [29]:
# Generator
def create_autoencoder(seq_length, depth, latent_dim):
    inputs0 = tf.keras.layers.Input(shape=(seq_length,1), name="Input0")
    x = tf.keras.layers.Flatten(name="flatten")(inputs0)
    x = tf.keras.layers.Dense(1024, activation="relu", name="Dense_0")(x)
    x = tf.keras.layers.Dense(512, activation="relu", name="Dense_1")(x)
    x = tf.keras.layers.Dense(256, activation="relu", name="Dense_2")(x)
    x = tf.keras.layers.Dense(latent_dim, activation="tanh", name="Dense_3")(x)
    x = tf.keras.layers.Dense(256, activation="relu", name="Dense_4")(x)
    x = tf.keras.layers.Dense(512, activation="relu", name="Dense_5")(x)
    x = tf.keras.layers.Dense(1024, activation="relu", name="Dense_6")(x)
    x = tf.keras.layers.Dense(10*seq_length, activation="relu", name="Dense_Reshape")(x)
    x = tf.keras.layers.Reshape((seq_length, 10), name="Reshape")(x)
    x = tf.keras.layers.Dense(depth, activation="softmax", name="Output")(x)
    model = tf.keras.models.Model(inputs=inputs0, outputs=x, name="Autoencoder")
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [3]:
aa_vocal = np.load("DreamWalker/model_weights/PepTV_vocal.npy")
pep_decoder = tf.keras.layers.StringLookup(vocabulary=aa_vocal[1:], invert=True, oov_token='')

In [10]:
def onehot2seq(onehot):
    seq = tf.math.argmax(onehot, axis=2)
    seq = pep_decoder(seq).numpy().astype('str')
    seq = ["".join(chars) for chars in seq]
    return seq

In [31]:
latent_dim = 100
seq_length = 40
depth = 43
np.random.seed(8701)
demo_noise = (np.random.rand(3, latent_dim)-0.5)*2
autoencoder = create_autoencoder(seq_length, depth, latent_dim)
path = "DreamWalker/data/processed_data/GAN"
pep = np.load(f"{path}/enc_uniprot.npz")["data"]
kf = KFold(n_splits=10, shuffle=True, random_state=0)
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for fold, (train_index, val_index) in enumerate(kf.split(pep)):
        pep_fold = pep[val_index]
        pep_fold = pep_fold.reshape(pep_fold.shape[0], 40, 1)
        # pep_fold = tf.one_hot(pep_fold, depth=43)
        autoencoder.fit(pep_fold, pep_fold, epochs=1, batch_size=22, verbose=1, validation_split=0.2)
        decoder = tf.keras.models.Model(inputs=autoencoder.layers[6].input, outputs=autoencoder.layers[-1].output)
        demo_seq = decoder(demo_noise)
        del decoder
        demo_seq = onehot2seq(demo_seq)
        print(demo_seq)

Epoch 1/3
3921/3921 [==============================] - 21s 5ms/step - loss: 2.4657 - accuracy: 0.2846 - val_loss: 2.3768 - val_accuracy: 0.2622
['MRRLLELLDLSRLARIIRESSEEELRILRLL', 'MRRLRLREDRERLDIRYRELIDEEIIRIDRRLIR', 'MLLLLSLIALLLLAL']
3921/3921 [==============================] - 19s 5ms/step - loss: 2.0373 - accuracy: 0.3405 - val_loss: 1.9208 - val_accuracy: 0.3441
['MVEELDSSRLSRLLRSRSDDLDDDIRSLSELE', 'MSELRVLRRRSRSERRRLDRSSDSSSDEEEELELLLR', 'MSLLVSISLSVLLLEP']
3921/3921 [==============================] - 19s 5ms/step - loss: 1.6136 - accuracy: 0.4357 - val_loss: 1.5860 - val_accuracy: 0.4409
['MSSLKDTESLTLLSRTVTDDVDDDLLSLELRDSE', 'VESSLVKKSVSSSSSRVKDKSSDSSSDSTSSSSELRKE', 'MTSLSTVSLSALKLSKKTVVAEVKVLRLEKREKE']
3921/3921 [==============================] - 19s 5ms/step - loss: 1.3794 - accuracy: 0.5124 - val_loss: 1.4074 - val_accuracy: 0.5021
['MVSSLDNDSRESLSLNSTDDSDDDLRSVDLLDSE', 'LNSSLVKKSISSSSSLVKDKSVDSTVDSYSSSSDKSKE', 'MNSLVSISSSRSKLEKTVAADVKVXLLLDKADRE']
3921/3921 [==============

In [32]:
demo_noise = (np.random.rand(20, latent_dim)-0.5)*2
decoder = tf.keras.models.Model(inputs=autoencoder.layers[6].input, outputs=autoencoder.layers[-1].output)
generated_seqs = decoder(demo_noise)
generated_seqs = onehot2seq(generated_seqs)
generated_seqs

['EDLLKFLLKELPLVSIKSEQDPNQTKKKNNTGKFK',
 'LNSLKDKKLGKADSASDYRSSDDPK',
 'MKLLLSKELKPLDLNCGYDHDVADPDPDDEEDEATLL',
 'LIVLDDELDPLTVVDFTSDIDQRIFSSV',
 'LTLLKKFKLSKLQKEDRAIGPCTRRFRASFDL',
 'DDTLKPDDLDDLDSDKDQDGDDGTDPL',
 'SDLLKKKLVNLLKVKNDFDFAARGESVDDATKLD',
 'KDLLKDDTLDPLVKSDDKPLGKPDDKINDKEILSDK',
 'KSLLDYSALDDLMDFTTTQDDPDDDPDSSIEDLD',
 'DNDLDYQVLIKSESQKSDYESRKDLLDFALI',
 'SSSLLMELKRDLNKEDENHDDSVDDFSDDK',
 'MKLLDDDLIKLSLEFSKDFAQSFKHCDPGDDTDVKLLA',
 'KKLLKDLKLTCDDSLHKFIFRCTFD',
 'LDELNGILDVLDLDSKYFSDKMDGDPDPDTPGDDDK',
 'STDLNRDDLKKLDKKDVTDGHDNEVEKHTLKP',
 'MKSLDDEDSDDQKRPAKYTLDGRDETSGDKFK',
 'DGLLDDNLDELDEDDKNVKLIDTPDDTANDADK',
 'LGDLDDSLDDLEGRLKDDFANTTDRFQL',
 'AKLLKTTKLPRLLEDNVHLDDDTKDGLVDLKLL',
 'EDLLDKNLKDSLDIYDSRDKENDVETDDSKK']

In [33]:
autoencoder.save("PeptideAutoencoder.keras")

In [ ]:
# np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_10",generated_seqs)

In [ ]:
#Convert to One_hot
# generated_seqs_one_hot = tf.one_hot(generated_seqs.squeeze(), depth=43)
# np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_one_hot_10",generated_seqs_one_hot)